# Analýza plánovaných hodin na učitele (ve skupině – např. katedra/fakulta) v daném časovém období (gql_ug + gql_events)
- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf

Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

<b>Instalace potřebných knihoven</b>

In [16]:
%pip install pandas aiohttp plotly.express asyncio nbformat aiofiles

In [21]:
import aiohttp
import asyncio
import json
import plotly.express as px
import pandas as pd
from itertools import product
from functools import reduce

<b>Funkce pro získání tokenu</b>


In [22]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            keyJson = await resp.json()

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            tokenJson = await resp.json()
    return tokenJson.get("token", None)

<b>Funkce pro definici GraphQL dotazu</b>

In [23]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

<b>Pomocné funkce pro zpracování dat</b>

In [24]:
def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

<b>Přihlašovací údaje</b>

In [25]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

<b>GraphQL dotaz</b>

In [26]:
queryStr = """
{
  groupPage {
    memberships {
      group {
        id
        name
      }
      user {
        id
        fullname
        events(
          where: {_and: [{startdate: {_ge: "2023-03-19T08:00:00"}}, {enddate: {_le: "2023-05-19T08:00:00"}}]}
        ) {
          id
          name
          startdate
          enddate
          presences {
            id
            invitationType {
              id
              name
            }
          }
        }
      }
    }
  }
}
"""

Tyhle dva se musí upravit pro náš projekt...struktura je od Martina

In [27]:
mappers = {
    "groupID": "memberships.group.id",
    "groupName": "memberships.group.name",
    "userID": "memberships.user.id",
    "userFullname": "memberships.user.fullname",
    "eventID": "memberships.user.events.id",
    "eventName": "memberships.user.events.name",
    "startdate": "memberships.user.events.startdate",
    "eventEndDate": "memberships.user.events.enddate",
    "presenceID": "memberships.user.events.presences.id",
    "invitationTypeID": "memberships.user.events.presences.invitationType.id",
    "invitationTypeName": "memberships.user.events.presences.invitationType.name",
    "presencesCount": "presencesCount",
}

In [28]:
async def fullPipe():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})
    print(response)
    data = response.get("data", None)
    
    # Debugging statement to inspect the structure
    print("Data:", data)
    
    # Check if data is not None and is a dictionary
    if not data or not isinstance(data, dict):
        raise ValueError("Invalid response data structure")
    
    groupPage = data.get("groupPage")
    
    # Ensure groupPage is a dictionary and extract memberships
    memberships = []
    if isinstance(groupPage, list) and len(groupPage) > 0:
        memberships = groupPage[0].get("memberships", [])
    elif isinstance(groupPage, dict):
        memberships = groupPage.get("memberships", [])

    # Flatten and map the data using list comprehensions
    resultMapped = []
    for membership in memberships:
        group = membership.get("group", {})
        user = membership.get("user", {})
        events = user.get("events", [])
        for event in events:
            presences = event.get("presences", [])
            for presence in presences:
                resultMapped.append({
                    "groupID": group.get("id"),
                    "groupName": group.get("name"),
                    "userID": user.get("id"),
                    "userFullname": user.get("fullname"),
                    "eventID": event.get("id"),
                    "eventName": event.get("name"),
                    "startdate": event.get("startdate"),
                    "eventEndDate": event.get("enddate"),
                    "presenceID": presence.get("id"),
                    "invitationTypeID": presence.get("invitationType", {}).get("id"),
                    "invitationTypeName": presence.get("invitationType", {}).get("name"),
                    "presencesCount": len(presences)
                })

    return resultMapped

async def main():
    flatData = await fullPipe()
    print(flatData)
    with open('resultNotebook.json', "w", encoding='utf-8') as outputFile:
        json.dump(flatData, outputFile)
        print("json Dumped")

await main()


{'data': {'groupPage': [{'memberships': [{'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'fullname': 'John Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dc868-a4a2-11ed-b9df-0242ac120003', 'fullname': 'Julia Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dc9a8-a4a2-11ed-b9df-0242ac120003', 'fullname': 'Johnson Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dcbec-a4a2-11ed-b9df-0242ac120003', 'fullname': 'Jepeto Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'fullname': 'Jana Newbie', 'events': [{'id': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'name': 'Zkouška', 'startdate': '2023-04-19T08:00:00', '

<h2>Tvorba tabulky</h2>

In [29]:
with open("resultNotebook.json", "r") as file: #pro reálné data nahradit resultNotebook.json
    data = json.load(file)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
df = pd.DataFrame(data)

print(df)

                                 groupID groupName                                userID userFullname                               eventID eventName            startdate         eventEndDate                            presenceID                      invitationTypeID invitationTypeName  presencesCount
0   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242ac110002  Jana Newbie  45b2df80-ae0f-11ed-9bd8-0242ac110002   Zkouška  2023-04-19T08:00:00  2023-04-19T09:00:00  89d1e684-ae0f-11ed-9bd8-0242ac110002  e871403c-a79c-11ed-b76e-0242ac110002             přijal              10
1   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242ac110002  Jana Newbie  45b2df80-ae0f-11ed-9bd8-0242ac110002   Zkouška  2023-04-19T08:00:00  2023-04-19T09:00:00  89d1f2d2-ae0f-11ed-9bd8-0242ac110002  e871403c-a79c-11ed-b76e-0242ac110002             přijal              10
2   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242ac110002  J

<h2>Tvorba grafů</h2>

In [30]:
import json
import aiofiles
from typing import List, Union, Dict
import asyncio

async def load_json_data(path: str) -> Union[List, None]:
    try:
        async with aiofiles.open(path, 'r', encoding='utf-8') as file:
            data = await file.read()
            return json.loads(data)
    except FileNotFoundError:
        print(f"Provided file path does not exist.")
        return None
    except Exception as e:
        print(f"Error {e}")
        return None

async def user_attendance(data: List[Dict]) -> List[Dict]:
    source_table = []
    is_teacher = "e8713b6e-a79c-11ed-b76e-0242ac110002"  # Replace with your specific user ID if needed
    is_specific_group ="groupIdOfK209"

    for item in data:
            if item['invitationTypeID'] == is_teacher:
                if item['groupID'] == is_specific_group:
                    row = {
                        "userID": item['userID'],
                        "userFullname": item['userFullname'],
                        "eventID": item['eventID'],
                        "eventName": item['eventName'],
                        "invitationTypeID": item['invitationTypeID'],
                        "invitationTypeName": item['invitationTypeName'],
                        "lessonLength": item['presencesCount'] * 90,  # in minutes
                    }
                    source_table.append(row)

    print(source_table)

    async with aiofiles.open('resultpokus.json', "w", encoding='utf-8') as output_file:
        await output_file.write(json.dumps(source_table, ensure_ascii=False, indent=4))

async def main():
    data = await load_json_data('resultFake.json')
    if data:
        await user_attendance(data)

# If running in an environment that already has an event loop
try:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        await main()  # Directly await main if the event loop is already running
    else:
        loop.run_until_complete(main())
except RuntimeError as e:
    if "This event loop is already running" in str(e):
        await main()  # Directly await main if the event loop is already running
    else:
        raise


[{'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'ABT', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'invitationTypeName': 'organizator', 'lessonLength': 900}, {'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'AIZ', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'invitationTypeName': 'organizator', 'lessonLength': 900}, {'userID': '29d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Karel Nekarel', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'INF', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'invitationTypeName': 'organizator', 'lessonLength': 1350}, {'userID': '29d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Ivan Pytel', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'ZaP', 'invitationTypeID': 'e8713b6

In [31]:
import json
import plotly.express as px

# JSON data
# TODOresult a fakedata upravit
with open('resultpokus.json', 'r') as file:
    data = json.load(file)

# Extract information for the pie chart
labels = [f"{item['userFullname']} - {item['eventName']}" for item in data]
sizes = [item['lessonLength'] for item in data]

# Create a pie chart using Plotly
fig = px.pie(values=sizes, names=labels, title='Lesson Length Distribution by User and Event')

# Show the pie chart
fig.show()


In [39]:
# JSON data
with open('resultFake.json', 'r') as file:
    data = json.load(file)

# Převod dat do pandas DataFrame
df = pd.DataFrame(data)

# Agregace dat pro získání celkového počtu 'presencesCount' podle 'groupID' a 'userFullname'
df_aggregated = df.groupby(['groupName', 'userFullname']).agg({'presencesCount': 'sum'}).reset_index()

# Vytvoření sunburst grafu
fig_sunburst = px.sunburst(
    df_aggregated,
    path=['groupName', 'userFullname', 'presencesCount'],
    values='presencesCount', 
    title='Přehled přítomnosti podle skupin a uživatelů'
)

# Aktualizace layoutu grafu
fig_sunburst.update_layout(
    width=800,  
    height=800 
)

fig_sunburst.show()

ValueError: Value of 'path_0' is not the name of a column in 'data_frame'. Expected one of ['groupName', 'userFullname', 'presencesCount'] but received: groupID